In [ ]:
import psutil
import torch

def estimate_training_parameters():
    # Get system specifications
    cpu_count = psutil.cpu_count(logical=False)
    total_memory_gb = psutil.virtual_memory().total / (1024 ** 3)
    gpu_count = torch.cuda.device_count()
    gpu_memory_gb = torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)

    # Estimate training parameters
    per_device_train_batch_size = min(8, cpu_count)
    gradient_accumulation_steps = max(1, min(16, int(cpu_count / per_device_train_batch_size)))
    optim = 'paged_adamw_32bit' if gpu_memory_gb >= 32 else 'adamw'
    learning_rate = 2e-4 if gpu_memory_gb >= 32 else 1e-4
    fp16 = gpu_memory_gb >= 16
    bf16 = gpu_memory_gb >= 16
    max_seq_length = min(1024, int(gpu_memory_gb * 0.5))
    num_train_epochs = min(3, int(total_memory_gb * 0.5))
    max_grad_norm = 0.3
    lr_scheduler_type = 'linear'

    # Return estimated parameters as a dictionary
    return {
        'per_device_train_batch_size': per_device_train_batch_size,
        'gradient_accumulation_steps': gradient_accumulation_steps,
        'optim': optim,
        'learning_rate': learning_rate,
        'fp16': fp16,
        'bf16': bf16,
        'max_seq_length': max_seq_length,
        'num_train_epochs': num_train_epochs,
        'max_grad_norm': max_grad_norm,
        'lr_scheduler_type': lr_scheduler_type
    }

# Example usage
estimated_params = estimate_training_parameters()
print(estimated_params)


```python
# Setting the training arguments
training_arguments_opt = TrainingArguments(
    output_dir="./results",
    num_train_epochs=estimated_params['num_train_epochs'],
    per_device_train_batch_size=estimated_params['per_device_train_batch_size'],
    gradient_accumulation_steps=estimated_params['gradient_accumulation_steps'],
    optim="paged_adamw_32bit",
    learning_rate=estimated_params['learning_rate'],
    fp16=estimated_params['fp16'],
    bf16=estimated_params['bf16'],
    max_grad_norm=estimated_params['max_grad_norm'],
    lr_scheduler_type=estimated_params['lr_scheduler_type']
)
```